In [1]:
from mapper_base import MapperBase
from astropy.io import fits
from astropy.table import Table

import mapper_DES
import pyccl as ccl
import numpy as np
import pylab as plt
import pymaster as nmt
import healpy as hp
import os

In [2]:
# Initialize binning scheme with 20 ells per bandpower
nside = 512 #4096
nside_mask = 512
bands = nmt.NmtBin.from_nside_linear(nside, 20)
ell_arr = bands.get_effective_ells()

In [3]:
DES_1_config = {'data_catalogs':['/home/zcapjru/PhD/Data/DES_redm/DES_Y1A1_3x2pt_redMaGiC_zerr_CATALOG.fits'], 
          'random_catalogs':['/home/zcapjru/PhD/Data/DES_redm/DES_Y1A1_3x2pt_redMaGiC_RANDOMS.fits'], 
                'file_mask':['/home/zcapjru/PhD/Data/DES_redm/DES_Y1A1_3x2pt_redMaGiC_MASK_HPIX4096RING.fits'],
                'nz_file':['/home/zcapjru/PhD/Data/DES_redm/2pt_NG_mcal_1110.fits'],
          'z_edges':[0.15, 0.30],
          'nside':nside, 'nside_mask': nside_mask }

DES_2_config = {'data_catalogs':['/home/zcapjru/PhD/Data/DES_redm/DES_Y1A1_3x2pt_redMaGiC_zerr_CATALOG.fits'], 
          'random_catalogs':['/home/zcapjru/PhD/Data/DES_redm/DES_Y1A1_3x2pt_redMaGiC_RANDOMS.fits'], 
                'file_mask':['/home/zcapjru/PhD/Data/DES_redm/DES_Y1A1_3x2pt_redMaGiC_MASK_HPIX4096RING.fits'],
                'nz_file':['/home/zcapjru/PhD/Data/DES_redm/2pt_NG_mcal_1110.fits'],
          'z_edges':[0.30, 0.45],
          'nside':nside, 'nside_mask': nside_mask }

DES_3_config = {'data_catalogs':['/home/zcapjru/PhD/Data/DES_redm/DES_Y1A1_3x2pt_redMaGiC_zerr_CATALOG.fits'], 
          'random_catalogs':['/home/zcapjru/PhD/Data/DES_redm/DES_Y1A1_3x2pt_redMaGiC_RANDOMS.fits'], 
                'file_mask':['/home/zcapjru/PhD/Data/DES_redm/DES_Y1A1_3x2pt_redMaGiC_MASK_HPIX4096RING.fits'],
                'nz_file':['/home/zcapjru/PhD/Data/DES_redm/2pt_NG_mcal_1110.fits'],
          'z_edges':[0.45, 0.60],
          'nside':nside, 'nside_mask': nside_mask }

DES_4_config = {'data_catalogs':['/home/zcapjru/PhD/Data/DES_redm/DES_Y1A1_3x2pt_redMaGiC_zerr_CATALOG.fits'], 
          'random_catalogs':['/home/zcapjru/PhD/Data/DES_redm/DES_Y1A1_3x2pt_redMaGiC_RANDOMS.fits'], 
                'file_mask':['/home/zcapjru/PhD/Data/DES_redm/DES_Y1A1_3x2pt_redMaGiC_MASK_HPIX4096RING.fits'],
                'nz_file':['/home/zcapjru/PhD/Data/DES_redm/2pt_NG_mcal_1110.fits'],
          'z_edges':[0.60, 0.75],
          'nside':nside, 'nside_mask': nside_mask }

DES_5_config = {'data_catalogs':['/home/zcapjru/PhD/Data/DES_redm/DES_Y1A1_3x2pt_redMaGiC_zerr_CATALOG.fits'], 
          'random_catalogs':['/home/zcapjru/PhD/Data/DES_redm/DES_Y1A1_3x2pt_redMaGiC_RANDOMS.fits'], 
                'file_mask':['/home/zcapjru/PhD/Data/DES_redm/DES_Y1A1_3x2pt_redMaGiC_MASK_HPIX4096RING.fits'],
                'nz_file':['/home/zcapjru/PhD/Data/DES_redm/2pt_NG_mcal_1110.fits'],
          'z_edges':[0.75, 0.90],
          'nside':nside, 'nside_mask': nside_mask }

In [4]:
DES_1_maps = mapper_DES.MapperDES(DES_1_config)
#DES_2_maps = mapper_DES.MapperDES(DES_2_config)
#DES_3_maps = mapper_DES.MapperDES(DES_3_config)
#DES_4_maps = mapper_DES.MapperDES(DES_4_config)
#DES_5_maps = mapper_DES.MapperDES(DES_5_config)

/home/zcapjru/anaconda3/envs/PhD/lib/python3.8/site-packages/healpy/fitsfunc.py:368: UserWarning: If you are not specifying the input dtype and using the default np.float64 dtype of read_map(), please consider that it will change in a future version to None as to keep the same dtype of the input file: please explicitly set the dtype if it is important to you.
  warnings.warn(


IndexError: boolean index did not match indexed array along dimension 0; dimension is 3145728 but corresponding boolean dimension is 201326592

In [ ]:
DES_nz_1 =DES_1_maps.get_nz()
DES_z_1 = DES_nz_1[1]
DES_dNdz_1 = DES_nz_1[0]

In [ ]:
plt.plot(DES_z_1, DES_dNdz_1, 'r-', label='Nz')

plt.xlabel('z', fontsize=16)
plt.ylabel('Nz', fontsize=16)
plt.legend(loc='upper left', ncol=2, labelspacing=0.1)
plt.show()

In [ ]:
def compute_cl(f_a, f_b, wsp):
    # Compute the power spectrum (a la anafast) of the masked fields
    # Note that we only use n_iter=0 here to speed up the computation,
    # but the default value of 3 is recommended in general.
    cl_coupled = nmt.compute_coupled_cell(f_a, f_b)
    # Decouple power spectrum into bandpowers inverting the coupling matrix
    cl_decoupled = wsp.decouple_cell(cl_coupled) #removed bias here

    return cl_decoupled

In [ ]:
# Read healpix maps and initialize a spin-0 and spin-2 field
DES_signal_1 = DES_1_maps.get_signal_map()
DES_mask_1  = DES_1_maps.get_mask()

In [ ]:
hp.visufunc.mollview(DES_signal_1, title= 'Mask')

In [ ]:
hp.visufunc.mollview(DES_mask_1, title= 'Mask')


In [ ]:
DES_f_1 =  DES_1_maps.get_nmt_field(DES_signal_1, DES_mask_1)

In [ ]:
DES_w_11 = nmt.NmtWorkspace()
DES_w_11.compute_coupling_matrix(DES_f_1, DES_f_1, bands)

In [ ]:
DES_empirical_11 = compute_cl(DES_f_1, DES_f_1, DES_w_11)

In [ ]:
SGC_empirical_11 = compute_cl(SGC_f_1, SGC_f_1, SGC_w_11)
SGC_empirical_12 = compute_cl(SGC_f_1, SGC_f_2, SGC_w_12)
SGC_empirical_22 = compute_cl(SGC_f_2, SGC_f_2, SGC_w_22)

In [ ]:
NGC_nl_11 = NGC_1_maps.get_nl_coupled() 
NGC_nl_22 = NGC_2_maps.get_nl_coupled() 

NGC_nl_11 = NGC_w_11.decouple_cell(NGC_nl_11)
NGC_nl_22 = NGC_w_22.decouple_cell(NGC_nl_22)

In [ ]:
SGC_nl_11 = SGC_1_maps.get_nl_coupled() 
SGC_nl_22 = SGC_2_maps.get_nl_coupled() 

SGC_nl_11 = SGC_w_11.decouple_cell(SGC_nl_11)
SGC_nl_22 = SGC_w_22.decouple_cell(SGC_nl_22)

In [ ]:
NGC_nn_empirical_11 = NGC_empirical_11 - NGC_nl_11
NGC_nn_empirical_22 = NGC_empirical_22 - NGC_nl_22

In [ ]:
SGC_nn_empirical_11 = SGC_empirical_11 - SGC_nl_11
SGC_nn_empirical_22 = SGC_empirical_22 - SGC_nl_22

In [ ]:
plt.plot(ell_arr, NGC_theory_11, 'r--', label='NGC Theory 11')
plt.plot(ell_arr, NGC_nn_empirical_11[0], 'r-', label='NGC Empirical 11')

plt.loglog()
plt.title('Quasars density angular power spectrum')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='lower left', ncol=2, labelspacing=0.1)
plt.show()

In [ ]:
plt.plot(ell_arr, NGC_theory_12, 'r--', label='NGC Theory 12')
plt.plot(ell_arr, NGC_empirical_12[0], 'r-', label='NGC Empirical 12')

plt.loglog()
plt.title('Quasars density angular power spectrum')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='lower left', ncol=2, labelspacing=0.1)
plt.show()

In [ ]:
plt.plot(ell_arr, NGC_theory_22, 'r--', label='NGC Theory 22')
plt.plot(ell_arr, NGC_nn_empirical_22[0], 'r-', label='NGC Empirical 22')

plt.loglog()
plt.title('Quasars density angular power spectrum')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='lower left', ncol=2, labelspacing=0.1)
plt.show()

In [ ]:
plt.plot(ell_arr, SGC_theory_11, 'b--', label='SGC Theory 11')
plt.plot(ell_arr, SGC_nn_empirical_11[0], 'b-', label='SGC Empirical 11')

plt.loglog()
plt.title('Quasars density angular power spectrum')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='lower left', ncol=2, labelspacing=0.1)
plt.show()

In [ ]:
plt.plot(ell_arr, SGC_theory_12, 'b--', label='SGC Theory 12')
plt.plot(ell_arr, SGC_empirical_12[0], 'b-', label='SGC Empirical 12')

plt.loglog()
plt.title('Quasars density angular power spectrum')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='lower left', ncol=2, labelspacing=0.1)
plt.show()

In [ ]:
plt.plot(ell_arr, SGC_theory_22, 'b--', label='SGC Theory 22')
plt.plot(ell_arr, SGC_nn_empirical_22[0], 'b-', label='SGC Empirical 22')

plt.loglog()
plt.title('Quasars density angular power spectrum')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='lower left', ncol=2, labelspacing=0.1)
plt.show()